# II Experimentation on effective data from philippines

In [1]:
# Initialisations
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 8) # (w, h)
from extraction import *
from preprocess import *
from parameters import *
from  model.MDBN import MDBN
import time
import pickle
import argparse
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn.datasets.samples_generator import make_blobs
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


## 1. Chargement des données.

In [2]:
data = {}
data[0] = np.load('.\\data\\phili2017\\features\\origin\\ar.npy'),np.load('.\\data\\phili2017\\labels\\labels_ar.npy')
data[1] = np.load('.\\data\\phili2017\\features\\origin\\si.npy'),np.load('.\\data\\phili2017\\labels\\labels_si.npy')
data[2] = np.load('.\\data\\phili2017\\features\\origin\\vv.npy'),np.load('.\\data\\phili2017\\labels\\labels_vv.npy')
data[3] = np.load('.\\data\\phili2017\\features\\origin\\sg.npy'),np.load('.\\data\\phili2017\\labels\\labels_sg.npy')

## 2. Definition des traces d'apprentissages du système

In [3]:
LS = ['Processing', 'Perception', 'Input', 'Understanding']

In [4]:
Classes = {} #Style d'apprentissage
Classes[0] = ["Active","Reflective"]
Classes[1] = ["Sensitive","Intuitive"]
Classes[2] = ["Visual","Verbal"] 
Classes[3] = ["Sequential","Global"] 
FSLSM = ["AR","SI","VV","SG"]

In [5]:
targets = {}
for i in range(0, len(data)):
    targets[i] = data[i][1]

In [6]:
for i in range(0, len(data)):
    nb1 = (targets[i]==1).sum()
    nb2 = (targets[i]==-1).sum()
    print ("Felder's Dimension: ",LS[i],"\t",Classes [i][0], ": ",nb1, "\n\t\t\t\t\t",Classes [i][1], ": ",nb2)

Felder's Dimension:  Processing 	 Active :  244 
					 Reflective :  263
Felder's Dimension:  Perception 	 Sensitive :  159 
					 Intuitive :  348
Felder's Dimension:  Input 	 Visual :  388 
					 Verbal :  119
Felder's Dimension:  Understanding 	 Sequential :  250 
					 Global :  257


## 3. Test data on dummy classifier

In [7]:
from sklearn.dummy import DummyClassifier

In [8]:
def evaluation_surclassifieur_naif(pos):
    X_train, X_test, Y_train, Y_test  = train_test_split(data[pos][0],targets[pos], test_size=0.2,random_state=0)
    # DummyClassifier to predict only target 0
    print("Classifieur Naif pour: ",LS[pos])
    dummy = DummyClassifier(strategy='most_frequent').fit(X_train, Y_train)
    dummy_pred = dummy.predict(X_test)

    # checking unique labels
    print('Style d\'apprentissage majoritaire prédit: ', Classes[pos][int(np.unique(dummy_pred))])

    # checking accuracy
    print('Test score: ', accuracy_score(Y_test, dummy_pred))

In [9]:
evaluation_surclassifieur_naif(0)

Classifieur Naif pour:  Processing
Style d'apprentissage majoritaire prédit:  Reflective
Test score:  0.5490196078431373


In [10]:
evaluation_surclassifieur_naif(1)

Classifieur Naif pour:  Perception
Style d'apprentissage majoritaire prédit:  Intuitive
Test score:  0.7941176470588235


In [11]:
evaluation_surclassifieur_naif(2)

Classifieur Naif pour:  Input
Style d'apprentissage majoritaire prédit:  Verbal
Test score:  0.7254901960784313


In [12]:
evaluation_surclassifieur_naif(3)

Classifieur Naif pour:  Understanding
Style d'apprentissage majoritaire prédit:  Global
Test score:  0.49019607843137253


### 4. Test sur modele

In [30]:
np.random.seed(1337)  # for reproducibility
def evaluation_model_MLSIM(pos):
    X_train, X_test, Y_train, Y_test  = train_test_split(data[pos][0],targets[pos], test_size=0.2,random_state=0)    
    digitized = discretize_learning_traces(X_train)
    X_train_feats = binarized_features2(digitized,4)
    print("Model MLSIM pour: ",LS[pos])
    n = np.shape(X_train_feats)[0]
    m = np.shape(X_train_feats)[1]
    print (n,m)
    print ("Nbr students:", n)
    print ("Learning traces dimension:",m)
    # construct DBN
    nhid_param2 = 17 #HYPERPARAMS.nb_hiden_node2 #nobre de noeuds de la deuxime couche cachée
    nhid_param1 = 12 #HYPERPARAMS.nb_hiden_node1 #nobre de noeuds de la premiere couche cachée
    #dbn = MDBN(input_data=X_train_feats, label=Y_train, input_size=m, hidden_layer_sizes=[nhid_param1, nhid_param2], output_size=1, batch_size=Training.batch_size, learning_rate=HYPERPARAMS.learning_rate,epochs=Training.epochs, C=HYPERPARAMS.C)
    dbn = MDBN(input_data=X_train_feats, label=Y_train, input_size=m, hidden_layer_sizes=[nhid_param1, nhid_param2], output_size=1, batch_size=50, learning_rate=0.39,epochs=700, C=1.0)    
    start = time.time()
    # pre-training (TrainUnsupervisedDBN)
    print ("Training the Hybrid Deep Belief Net model \n................")
    dbn.train(X_train_feats)
    print ("Unsupervised training time: ",(time.time()-start)," seconds")
    # fine-tuning (DBNSupervisedFineTuning)
    synthetic_V_list, h_list = dbn.predict(X_train_feats)
    #finetune training the last layer
    dbn.finetune(X_train_feats,Y_train,"Train data",bytraining=True)
    digitized_test = discretize_learning_traces(X_test)
    X_test_feats = binarized_features2(digitized_test,4)
    accuracy = dbn.evaluate(X_test_feats,Y_test,"Test data",verbose=True)    

In [31]:
evaluation_model_MLSIM(0)

bins  [0, 1, 2, 3]  digitized 2  [[4 4 4 4]
 [4 4 3 4]]  mat abs  [[11  8  8  9]
 [ 4 10  2 35]]
Model MLSIM pour:  Processing
405 16
Nbr students: 405
Learning traces dimension: 16
Initialize weigths [[-2.97469156e-02 -4.26645035e-02 -2.77341851e-02 -5.08538910e-03
  -2.23749324e-02  2.29910257e-03 -2.97571343e-02  5.95106606e-02
   2.91018191e-02 -4.80907217e-02 -1.42156164e-02  1.60626474e-02
  -4.68677592e-02  6.04435756e-02 -7.09689142e-03  3.61947928e-02]
 [ 3.67648220e-02 -1.73423034e-02 -1.04870076e-02  1.05322660e-02
   3.25214717e-02 -3.90239493e-02 -2.64791065e-02  2.12773573e-02
  -4.39673285e-05 -4.01789154e-02 -1.08573373e-02 -3.76005956e-02
   3.96242545e-03  4.15463379e-02 -3.93436319e-02  5.71699030e-02]
 [-9.32316686e-03  5.00879895e-04  1.30886835e-03 -6.05260685e-02
   2.89612586e-02  6.16631300e-02 -4.21403083e-02 -4.66706525e-02
  -1.56457269e-02  2.41524305e-02 -6.21373712e-02 -1.63463672e-02
  -5.51650843e-02  3.61670106e-02 -1.87788484e-02  2.53154650e-02]
 [-1

>> Epoch 40 finished 	RBM 1 Reconstruction error 4.247605
>> Epoch 41 finished 	RBM 1 Reconstruction error 4.242826
>> Epoch 42 finished 	RBM 1 Reconstruction error 4.242145
>> Epoch 43 finished 	RBM 1 Reconstruction error 4.246675
>> Epoch 44 finished 	RBM 1 Reconstruction error 4.235581
>> Epoch 45 finished 	RBM 1 Reconstruction error 4.238031
>> Epoch 46 finished 	RBM 1 Reconstruction error 4.236667
>> Epoch 47 finished 	RBM 1 Reconstruction error 4.224459
>> Epoch 48 finished 	RBM 1 Reconstruction error 4.223295
>> Epoch 49 finished 	RBM 1 Reconstruction error 4.215855
>> Epoch 50 finished 	RBM 1 Reconstruction error 4.220570
>> Epoch 51 finished 	RBM 1 Reconstruction error 4.209882
>> Epoch 52 finished 	RBM 1 Reconstruction error 4.208070
>> Epoch 53 finished 	RBM 1 Reconstruction error 4.216374
>> Epoch 54 finished 	RBM 1 Reconstruction error 4.215409
>> Epoch 55 finished 	RBM 1 Reconstruction error 4.196302
>> Epoch 56 finished 	RBM 1 Reconstruction error 4.194755
>> Epoch 57 fi

>> Epoch 183 finished 	RBM 1 Reconstruction error 2.459893
>> Epoch 184 finished 	RBM 1 Reconstruction error 2.437849
>> Epoch 185 finished 	RBM 1 Reconstruction error 2.422376
>> Epoch 186 finished 	RBM 1 Reconstruction error 2.414282
>> Epoch 187 finished 	RBM 1 Reconstruction error 2.414492
>> Epoch 188 finished 	RBM 1 Reconstruction error 2.407606
>> Epoch 189 finished 	RBM 1 Reconstruction error 2.390517
>> Epoch 190 finished 	RBM 1 Reconstruction error 2.384137
>> Epoch 191 finished 	RBM 1 Reconstruction error 2.375250
>> Epoch 192 finished 	RBM 1 Reconstruction error 2.367086
>> Epoch 193 finished 	RBM 1 Reconstruction error 2.346710
>> Epoch 194 finished 	RBM 1 Reconstruction error 2.347856
>> Epoch 195 finished 	RBM 1 Reconstruction error 2.353863
>> Epoch 196 finished 	RBM 1 Reconstruction error 2.344797
>> Epoch 197 finished 	RBM 1 Reconstruction error 2.328505
>> Epoch 198 finished 	RBM 1 Reconstruction error 2.308612
>> Epoch 199 finished 	RBM 1 Reconstruction error 2.3030

>> Epoch 323 finished 	RBM 1 Reconstruction error 1.377361
>> Epoch 324 finished 	RBM 1 Reconstruction error 1.362370
>> Epoch 325 finished 	RBM 1 Reconstruction error 1.356412
>> Epoch 326 finished 	RBM 1 Reconstruction error 1.349382
>> Epoch 327 finished 	RBM 1 Reconstruction error 1.347837
>> Epoch 328 finished 	RBM 1 Reconstruction error 1.334322
>> Epoch 329 finished 	RBM 1 Reconstruction error 1.327126
>> Epoch 330 finished 	RBM 1 Reconstruction error 1.330868
>> Epoch 331 finished 	RBM 1 Reconstruction error 1.324979
>> Epoch 332 finished 	RBM 1 Reconstruction error 1.319204
>> Epoch 333 finished 	RBM 1 Reconstruction error 1.309406
>> Epoch 334 finished 	RBM 1 Reconstruction error 1.299058
>> Epoch 335 finished 	RBM 1 Reconstruction error 1.288473
>> Epoch 336 finished 	RBM 1 Reconstruction error 1.281957
>> Epoch 337 finished 	RBM 1 Reconstruction error 1.273746
>> Epoch 338 finished 	RBM 1 Reconstruction error 1.271630
>> Epoch 339 finished 	RBM 1 Reconstruction error 1.2664

>> Epoch 462 finished 	RBM 1 Reconstruction error 0.843349
>> Epoch 463 finished 	RBM 1 Reconstruction error 0.836598
>> Epoch 464 finished 	RBM 1 Reconstruction error 0.834839
>> Epoch 465 finished 	RBM 1 Reconstruction error 0.834777
>> Epoch 466 finished 	RBM 1 Reconstruction error 0.830738
>> Epoch 467 finished 	RBM 1 Reconstruction error 0.828360
>> Epoch 468 finished 	RBM 1 Reconstruction error 0.826532
>> Epoch 469 finished 	RBM 1 Reconstruction error 0.826522
>> Epoch 470 finished 	RBM 1 Reconstruction error 0.826106
>> Epoch 471 finished 	RBM 1 Reconstruction error 0.824466
>> Epoch 472 finished 	RBM 1 Reconstruction error 0.822952
>> Epoch 473 finished 	RBM 1 Reconstruction error 0.822135
>> Epoch 474 finished 	RBM 1 Reconstruction error 0.819525
>> Epoch 475 finished 	RBM 1 Reconstruction error 0.818751
>> Epoch 476 finished 	RBM 1 Reconstruction error 0.820576
>> Epoch 477 finished 	RBM 1 Reconstruction error 0.820794
>> Epoch 478 finished 	RBM 1 Reconstruction error 0.8179

...

>> Epoch 43 finished 	RBM 2 Reconstruction error 5.396615
>> Epoch 44 finished 	RBM 2 Reconstruction error 5.376103
>> Epoch 45 finished 	RBM 2 Reconstruction error 5.379669
>> Epoch 46 finished 	RBM 2 Reconstruction error 5.353236
>> Epoch 47 finished 	RBM 2 Reconstruction error 5.332143
>> Epoch 48 finished 	RBM 2 Reconstruction error 5.334403
>> Epoch 49 finished 	RBM 2 Reconstruction error 5.317031
>> Epoch 50 finished 	RBM 2 Reconstruction error 5.288430
>> Epoch 51 finished 	RBM 2 Reconstruction error 5.269408
>> Epoch 52 finished 	RBM 2 Reconstruction error 5.242105
>> Epoch 53 finished 	RBM 2 Reconstruction error 5.234835
>> Epoch 54 finished 	RBM 2 Reconstruction error 5.205307
>> Epoch 55 finished 	RBM 2 Reconstruction error 5.190594
>> Epoch 56 finished 	RBM 2 Reconstruction error 5.152927
>> Epoch 57 finished 	RBM 2 Reconstruction error 5.134927
>> Epoch 58 finished 	RBM 2 Reconstruction error 5.104893
>> Epoch 59 finished 	RBM 2 Reconstruction error 5.076617
>> Epoch 60 fi

>> Epoch 182 finished 	RBM 2 Reconstruction error 3.178668
>> Epoch 183 finished 	RBM 2 Reconstruction error 3.179010
>> Epoch 184 finished 	RBM 2 Reconstruction error 3.176815
>> Epoch 185 finished 	RBM 2 Reconstruction error 3.192047
>> Epoch 186 finished 	RBM 2 Reconstruction error 3.160937
>> Epoch 187 finished 	RBM 2 Reconstruction error 3.151904
>> Epoch 188 finished 	RBM 2 Reconstruction error 3.149967
>> Epoch 189 finished 	RBM 2 Reconstruction error 3.139866
>> Epoch 190 finished 	RBM 2 Reconstruction error 3.129016
>> Epoch 191 finished 	RBM 2 Reconstruction error 3.126687
>> Epoch 192 finished 	RBM 2 Reconstruction error 3.126228
>> Epoch 193 finished 	RBM 2 Reconstruction error 3.118016
>> Epoch 194 finished 	RBM 2 Reconstruction error 3.121024
>> Epoch 195 finished 	RBM 2 Reconstruction error 3.125081
>> Epoch 196 finished 	RBM 2 Reconstruction error 3.116583
>> Epoch 197 finished 	RBM 2 Reconstruction error 3.094458
>> Epoch 198 finished 	RBM 2 Reconstruction error 3.0947

>> Epoch 323 finished 	RBM 2 Reconstruction error 2.496343
>> Epoch 324 finished 	RBM 2 Reconstruction error 2.488678
>> Epoch 325 finished 	RBM 2 Reconstruction error 2.501412
>> Epoch 326 finished 	RBM 2 Reconstruction error 2.498646
>> Epoch 327 finished 	RBM 2 Reconstruction error 2.493813
>> Epoch 328 finished 	RBM 2 Reconstruction error 2.464375
>> Epoch 329 finished 	RBM 2 Reconstruction error 2.466283
>> Epoch 330 finished 	RBM 2 Reconstruction error 2.459715
>> Epoch 331 finished 	RBM 2 Reconstruction error 2.466800
>> Epoch 332 finished 	RBM 2 Reconstruction error 2.461832
>> Epoch 333 finished 	RBM 2 Reconstruction error 2.454694
>> Epoch 334 finished 	RBM 2 Reconstruction error 2.452665
>> Epoch 335 finished 	RBM 2 Reconstruction error 2.457066
>> Epoch 336 finished 	RBM 2 Reconstruction error 2.461174
>> Epoch 337 finished 	RBM 2 Reconstruction error 2.450698
>> Epoch 338 finished 	RBM 2 Reconstruction error 2.462772
>> Epoch 339 finished 	RBM 2 Reconstruction error 2.4534

>> Epoch 463 finished 	RBM 2 Reconstruction error 2.332066
>> Epoch 464 finished 	RBM 2 Reconstruction error 2.314404
>> Epoch 465 finished 	RBM 2 Reconstruction error 2.315059
>> Epoch 466 finished 	RBM 2 Reconstruction error 2.304756
>> Epoch 467 finished 	RBM 2 Reconstruction error 2.313141
>> Epoch 468 finished 	RBM 2 Reconstruction error 2.318867
>> Epoch 469 finished 	RBM 2 Reconstruction error 2.299824
>> Epoch 470 finished 	RBM 2 Reconstruction error 2.323038
>> Epoch 471 finished 	RBM 2 Reconstruction error 2.321700
>> Epoch 472 finished 	RBM 2 Reconstruction error 2.309654
>> Epoch 473 finished 	RBM 2 Reconstruction error 2.308787
>> Epoch 474 finished 	RBM 2 Reconstruction error 2.325873
>> Epoch 475 finished 	RBM 2 Reconstruction error 2.325767
>> Epoch 476 finished 	RBM 2 Reconstruction error 2.339523
>> Epoch 477 finished 	RBM 2 Reconstruction error 2.305764
>> Epoch 478 finished 	RBM 2 Reconstruction error 2.317911
>> Epoch 479 finished 	RBM 2 Reconstruction error 2.3015

>> Epoch 602 finished 	RBM 2 Reconstruction error 2.223869
>> Epoch 603 finished 	RBM 2 Reconstruction error 2.229879
>> Epoch 604 finished 	RBM 2 Reconstruction error 2.214826
>> Epoch 605 finished 	RBM 2 Reconstruction error 2.228662
>> Epoch 606 finished 	RBM 2 Reconstruction error 2.229837
>> Epoch 607 finished 	RBM 2 Reconstruction error 2.227311
>> Epoch 608 finished 	RBM 2 Reconstruction error 2.220890
>> Epoch 609 finished 	RBM 2 Reconstruction error 2.216451
>> Epoch 610 finished 	RBM 2 Reconstruction error 2.227260
>> Epoch 611 finished 	RBM 2 Reconstruction error 2.222503
>> Epoch 612 finished 	RBM 2 Reconstruction error 2.213260
>> Epoch 613 finished 	RBM 2 Reconstruction error 2.200500
>> Epoch 614 finished 	RBM 2 Reconstruction error 2.226693
>> Epoch 615 finished 	RBM 2 Reconstruction error 2.221353
>> Epoch 616 finished 	RBM 2 Reconstruction error 2.216184
>> Epoch 617 finished 	RBM 2 Reconstruction error 2.231629
>> Epoch 618 finished 	RBM 2 Reconstruction error 2.2264

In [15]:
evaluation_model_MLSIM(1)

bins  [0, 1, 2, 3]  digitized 2  [[4 4 4 4]
 [4 3 3 4]]  mat abs  [[10 10  5  6]
 [ 5  2  2  5]]
Model MLSIM pour:  Perception
405 9
Nbr students: 405
Learning traces dimension: 9
Initialize weigths [[ 0.08445475  0.04839338  0.1041746   0.00837779  0.00043698  0.00276273
   0.02109294 -0.09060167 -0.0561663 ]
 [ 0.04524727  0.00858941 -0.00652396 -0.04225748 -0.103246    0.02828774
  -0.08800544  0.05958094  0.08059222]
 [ 0.01782571  0.04744115 -0.0426989   0.0263591   0.04682413  0.0906379
   0.09374891 -0.00351502  0.09082323]
 [-0.06883402  0.09702214  0.06660083  0.02530898 -0.0506772  -0.07718517
   0.03746995  0.06590624  0.04341451]
 [ 0.03026961 -0.10439247  0.06432098  0.09658943  0.1011703  -0.08741697
   0.00044436  0.05036616  0.02511272]
 [ 0.04186632  0.11018115 -0.01378733  0.10527495 -0.11092328  0.03575614
   0.07960883 -0.09022406  0.01561606]
 [-0.10145339  0.07226221 -0.09431854  0.03465013  0.08847466  0.03289791
   0.07679725 -0.08618956  0.0874368 ]
 [ 0.074119

E:\paperResearch\MLSIM\extraction.py:21: RuntimeWarning: Mean of empty slice.
  bin_means = [mat_abs[digitized == i].mean() for i in range(1, len(bins))]


>> Epoch 1 finished 	RBM 1 Reconstruction error 3.166246
>> Epoch 2 finished 	RBM 1 Reconstruction error 2.220307
>> Epoch 3 finished 	RBM 1 Reconstruction error 1.962497
>> Epoch 4 finished 	RBM 1 Reconstruction error 1.850679
>> Epoch 5 finished 	RBM 1 Reconstruction error 1.811368
>> Epoch 6 finished 	RBM 1 Reconstruction error 1.795723
>> Epoch 7 finished 	RBM 1 Reconstruction error 1.784030
>> Epoch 8 finished 	RBM 1 Reconstruction error 1.776670
>> Epoch 9 finished 	RBM 1 Reconstruction error 1.770678
>> Epoch 10 finished 	RBM 1 Reconstruction error 1.768075
>> Epoch 11 finished 	RBM 1 Reconstruction error 1.768444
>> Epoch 12 finished 	RBM 1 Reconstruction error 1.765416
>> Epoch 13 finished 	RBM 1 Reconstruction error 1.765196
>> Epoch 14 finished 	RBM 1 Reconstruction error 1.764546
>> Epoch 15 finished 	RBM 1 Reconstruction error 1.763835
>> Epoch 16 finished 	RBM 1 Reconstruction error 1.762030
>> Epoch 17 finished 	RBM 1 Reconstruction error 1.762676
>> Epoch 18 finished 	R

>> Epoch 45 finished 	RBM 2 Reconstruction error 5.367043
>> Epoch 46 finished 	RBM 2 Reconstruction error 5.361541
>> Epoch 47 finished 	RBM 2 Reconstruction error 5.365348
>> Epoch 48 finished 	RBM 2 Reconstruction error 5.360237
>> Epoch 49 finished 	RBM 2 Reconstruction error 5.359085
>> Epoch 50 finished 	RBM 2 Reconstruction error 5.361549
>> Epoch 51 finished 	RBM 2 Reconstruction error 5.358977
>> Epoch 52 finished 	RBM 2 Reconstruction error 5.359929
>> Epoch 53 finished 	RBM 2 Reconstruction error 5.361602
>> Epoch 54 finished 	RBM 2 Reconstruction error 5.358598
>> Epoch 55 finished 	RBM 2 Reconstruction error 5.362124
>> Epoch 56 finished 	RBM 2 Reconstruction error 5.362683
>> Epoch 57 finished 	RBM 2 Reconstruction error 5.363968
>> Epoch 58 finished 	RBM 2 Reconstruction error 5.357890
>> Epoch 59 finished 	RBM 2 Reconstruction error 5.359038
>> Epoch 60 finished 	RBM 2 Reconstruction error 5.361931
>> Epoch 61 finished 	RBM 2 Reconstruction error 5.366724
>> Epoch 62 fi